In [1]:
all_positive_species = ["Anthocharis_cardamines","Aporia_crataegi","Argynnis_bischoffii_washingtonia","Aricia_agestis","Aricia_artaxerxes","Battus_philenor","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Calycopis_cecrops","Celastrina_argiolus","Coenonympha_glycerion","Colias_nastes","Colias_palaeno","Cyaniris_semiargus","Danaus_plexippus","Erebia_aethiops","Erebia_ligea","Euphydryas_editha","Fabriciana_adippe","Glaucopsyche_alexis","Hipparchia_semele","Lasiommata_megera","Leptidea_juvernica","Leptidea_reali","Leptidea_sinapis","Lysandra_bellargus","Lysandra_coridon","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Melitaea_cinxia","Mellicta_athalia","Nymphalis_c-album","Nymphalis_io","Nymphalis_polychloros","Nymphalis_urticae","Oeneis_ivallda","Papilio_glaucus","Papilio_machaon","Papilio_xuthus","Pararge_aegeria","Parnassius_glacialis","Phengaris_arion","Pieris_brassicae","Pieris_mannii","Pieris_melete","Pieris_napi","Pieris_rapae","Plebejus_argus","Polyommatus_icarus","Polyommatus_iphigenia"]

import pandas as pd
location = "/mnt/c/Users/sauba/Desktop/20.RERConverge_trimmed/3.New_colias/61.New_alignment_colias"
df = pd.read_csv(f"{location}/20.CLassification/rer_combined.csv")

# make a copy so original df is untouched
df_clean = df.copy()

# set geneid as index
df_clean = df_clean.set_index("Unnamed: 0")

# remove columns that are fully NA
df_clean = df_clean.dropna(axis=1, how="all")
# 

# clean species names (remove trailing .1, .2, etc.)
df_clean.columns = df_clean.columns.str.replace(r"\.\d+$", "", regex=True)

# merge duplicate species by summing (ignores NaNs)
df_clean = df_clean.groupby(df_clean.columns, axis=1).sum(min_count=1)

# optional: reset index if you want geneid back as a column
df_clean = df_clean.reset_index().rename(columns={"Unnamed: 0": "geneid"})
df_clean = df_clean.drop(columns=["NA"], errors="ignore")
df = df_clean.set_index("geneid").T

labels = {}
pos_lable = []
for species_name in df.index:
    if species_name in all_positive_species:
        labels[species_name] = 1
    else:
        labels[species_name] = 0
        # pos_lable.append(species_name)
# print(len(pos_lable),len(positive_species))
# labels
# for speciesName in all_positive_species:
#     if speciesName not in labels:
#         print(speciesName, "Missing")
df["target"] = df.index.map(labels)
df_full = df.copy()
print(len(labels))

89


/tmp/ipykernel_5048/1321376285.py:21: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_clean = df_clean.groupby(df_clean.columns, axis=1).sum(min_count=1)


In [43]:
def remove_species(large_list, small_list):
    out_list = []
    for species in large_list:
        if species not in small_list:
            out_list.append(species)
    return(out_list)
species_truth_dictionary = {}
correct_pred_percentage_list = []

for i in range(1000):
    print("run ", i)
    

    positive_species = []
    negative_species = []
    for species in labels:
        if labels[species] == 1:
            positive_species.append(species)
        else:
            negative_species.append(species)
    # print(len(positive_species))
    # print(len(negative_species))
    
    import random
    # random.seed(42)
    train_pos_count = int(len(positive_species)*2/3)
    train_neg_count = int(len(negative_species)*2/3)
    
    training_set_pos = (random.sample(positive_species,train_pos_count ))
    rest_pos = remove_species(positive_species,training_set_pos)
    
    training_set_neg = (random.sample(negative_species,train_neg_count ))
    rest_neg = remove_species(negative_species,training_set_neg)
    
    training_set = training_set_pos+training_set_neg
    test_set = rest_pos+rest_neg

    df = df_full.loc[df_full.index.isin(training_set)]


    import pandas as pd
    import numpy as np
    from sklearn.impute import KNNImputer
    from sklearn.pipeline import Pipeline
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import StratifiedKFold, cross_val_predict
    from sklearn.metrics import f1_score, classification_report, confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    import numpy as np
    from sklearn.feature_selection import SelectKBest, f_classif
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import GridSearchCV, StratifiedKFold
    import xgboost as xgb
    
    # --- 1. Features and labels ---
    X = df.drop(columns="target")
    y = df["target"]
    
    # --- 2. Define pipeline: KNN Imputer + Random Forest ---
    rf_pipeline = Pipeline([
        ("imputer", KNNImputer(n_neighbors=5, weights="uniform")),
        ("rf", RandomForestClassifier(
            n_estimators=1000,
            max_depth=None,
            max_features="sqrt",
            bootstrap=True,
            class_weight="balanced",  # optional, improves minority class recall
            random_state=42,
            n_jobs=-1
        ))
    ])
    # rf_pipeline = xgb.XGBClassifier(
    # n_estimators=500,
    # max_depth=5,
    # learning_rate=0.05,
    # subsample=0.8,
    # colsample_bytree=0.8,
    # random_state=42,
    # use_label_encoder=False,
    # eval_metric="logloss"
    # )

    
    # --- 3. Cross-validated predicted probabilities ---
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    y_proba = cross_val_predict(rf_pipeline, X, y, cv=cv, method="predict_proba")
    
    # --- 4. Search for best threshold ---
    # thresholds = np.linspace(0.1, 0.9, 81)
    # f1_scores = []
    
    # for t in thresholds:
    #     y_pred = (y_proba[:,1] >= t).astype(int)
    #     f1_scores.append(f1_score(y, y_pred))
    
    # best_idx = np.argmax(f1_scores)
    # best_threshold = thresholds[best_idx]
    best_threshold = 0.5
    # print(f"Best threshold for max F1: {best_threshold:.2f} (F1={f1_scores[best_idx]:.3f})")
    
    # # --- 5. Final predictions ---
    # y_pred_best = (y_proba[:,1] >= best_threshold).astype(int)
    
    # # --- 6. Classification report ---
    # print(classification_report(y, y_pred_best, digits=3))
    
    # # --- 7. Confusion matrix ---
    # cm = confusion_matrix(y, y_pred_best)
    # plt.figure(figsize=(6,5))
    # sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
    #             xticklabels=np.unique(y),
    #             yticklabels=np.unique(y))
    # plt.xlabel("Predicted")
    # plt.ylabel("True")
    # plt.title(f"Confusion Matrix (Best Threshold={best_threshold:.2f})")
    # plt.show()
    
    # # --- 8. Plot F1 vs threshold ---
    # plt.figure(figsize=(6,4))
    # plt.plot(thresholds, f1_scores, marker='o')
    # plt.axvline(best_threshold, color='red', linestyle='--', label=f'Best threshold={best_threshold:.2f}')
    # plt.xlabel("Threshold for class 1")
    # plt.ylabel("F1 Score")
    # plt.title("F1 Score vs Threshold")
    # plt.legend()
    # plt.grid(True)
    # plt.show()
    
    # --- 9. Fit RF on full data to get feature importances ---
    rf_pipeline.fit(X, y)
    # importances = rf_pipeline.named_steps['rf'].feature_importances_
    # feature_names = X.columns
    
    # # --- 10. Top 20 important genes ---
    # importance_df = pd.DataFrame({
    #     'Gene': feature_names,
    #     'Importance': importances
    # }).sort_values(by='Importance', ascending=False).head(20)
    
    # print("Top 20 important genes:")
    # print(importance_df)
    
    # # Optional: barplot of top 20 genes
    # plt.figure(figsize=(8,6))
    # sns.barplot(x='Importance', y='Gene', data=importance_df, palette='viridis')
    # plt.title("Top 20 Important Genes (Random Forest)")
    # plt.xlabel("Importance")
    # plt.ylabel("Gene")
    # plt.tight_layout()
    # plt.show()

    new_df = df_full.loc[df_full.index.isin(test_set)] 
    df_100 = new_df[X.columns] 
    X_new = df_100[X.columns]
    y_proba_new = rf_pipeline.predict_proba(X_new)
    y_pred_new = (y_proba_new[:, 1] >= best_threshold).astype(int)
    results = pd.DataFrame({
        "species": df_100.index,
        "predicted_class": y_pred_new,
        "prob_negative": y_proba_new[:, 0],
        "prob_positive": y_proba_new[:, 1]
        })
    count_true = 0
    count_false = 0
    for lines in results.iterrows():
        
        if lines[1].species not in species_truth_dictionary:
            species_truth_dictionary[lines[1].species] = []
        if max(lines[1].prob_negative,lines[1].prob_positive) - best_threshold > 0.1:
            if (lines[1].predicted_class) == 1 and (lines[1].species) in  all_positive_species:
                    # print(lines[1].species, "true")\
                species_truth_dictionary[lines[1].species].append(1)
                count_true += 1
            elif (lines[1].predicted_class) == 0 and (lines[1].species) not in  all_positive_species:
                # print(lines[1].species, "true negative", (lines[1].prob_negative) - (lines[1].prob_positive))
                count_true += 1
                species_truth_dictionary[lines[1].species].append(1)
            else:
                # print(lines[1].species, "False", (lines[1].prob_negative) - (lines[1].prob_positive))
                count_false += 1
                species_truth_dictionary[lines[1].species].append(0)
        else:
            print(lines[1].species,lines[1].prob_negative,lines[1].prob_positive,best_threshold)
            species_truth_dictionary[lines[1].species].append("c")
    correct_pred_percentage_list.append((count_true/(count_true+count_false)))
    print((count_true/(count_true+count_false)))
    print(((count_true,count_false,len(df_100.index))))

        

run  0
Aporia_crataegi 0.427 0.573 0.5
Argynnis_bischoffii_washingtonia 0.444 0.556 0.5
Battus_philenor 0.446 0.554 0.5
Boloria_selene 0.451 0.549 0.5
Colias_croceus 0.444 0.556 0.5
Colias_palaeno 0.463 0.537 0.5
Danaus_plexippus 0.479 0.521 0.5
Dircenna_loreta 0.43 0.57 0.5
Dryadula_phaetusa 0.554 0.446 0.5
Eueides_isabella 0.496 0.504 0.5
Eurema_hecabe 0.479 0.521 0.5
Leptidea_sinapis 0.465 0.535 0.5
Mechanitis_mazaeus 0.43 0.57 0.5
Napeogenes_inachia 0.418 0.582 0.5
Napeogenes_sylphis 0.462 0.538 0.5
Ornithoptera_alexandrae 0.424 0.576 0.5
Papilio_dardanus_tibullus 0.439 0.561 0.5
Papilio_demoleus 0.425 0.575 0.5
Papilio_machaon 0.402 0.598 0.5
Pieris_rapae 0.444 0.556 0.5
Teinopalpus_imperialis 0.472 0.528 0.5
Zerene_cesonia 0.411 0.589 0.5
1.0
(9, 0, 31)
run  1
Aporia_crataegi 0.494 0.506 0.5
Battus_philenor 0.465 0.535 0.5
Brenthis_hecate 0.501 0.499 0.5
Brenthis_ino 0.541 0.459 0.5
Colias_croceus 0.559 0.441 0.5
Danaus_chrysippus 0.495 0.505 0.5
Danaus_plexippus 0.472 0.528 0.5



KeyboardInterrupt



In [52]:
species_truth_dictionary = {}
correct_pred_percentage_list = []

from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.impute import KNNImputer
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import random

for i in range(1000):
    print("run ", i)

    # --- Split positive and negative species ---
    positive_species = [s for s in labels if labels[s] == 1]
    negative_species = [s for s in labels if labels[s] == 0]

    train_pos_count = int(len(positive_species) * 2 / 3)
    train_neg_count = int(len(negative_species) * 2 / 3)

    training_set_pos = random.sample(positive_species, train_pos_count)
    rest_pos = remove_species(positive_species, training_set_pos)

    training_set_neg = random.sample(negative_species, train_neg_count)
    rest_neg = remove_species(negative_species, training_set_neg)

    training_set = training_set_pos + training_set_neg
    test_set = rest_pos + rest_neg

    df = df_full.loc[df_full.index.isin(training_set)]

    # --- Features and labels ---
    X = df.drop(columns="target")
    y = df["target"]

    # --- Define diverse base learners with imputers ---
    estimators = [
        ("rf", make_pipeline(
            KNNImputer(n_neighbors=5),
            RandomForestClassifier(
                n_estimators=500,
                class_weight="balanced",
                random_state=42,
                n_jobs=-1
            )
        )),
        ("xgb", make_pipeline(
            KNNImputer(n_neighbors=5),
            xgb.XGBClassifier(
                n_estimators=500,
                max_depth=5,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                eval_metric="logloss",
                random_state=42,
                use_label_encoder=False,
                n_jobs=-1
            )
        )),
        ("svm", make_pipeline(
            KNNImputer(n_neighbors=5),
            StandardScaler(),
            CalibratedClassifierCV(
                estimator=LinearSVC(class_weight="balanced", max_iter=5000),
                method="sigmoid",  # Platt scaling
                cv=5
            )
        ))
    ]

    # --- Stacking meta-classifier ---
    stack = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression(max_iter=2000, class_weight="balanced"),
        stack_method="predict_proba",
        n_jobs=-1
    )

    # --- Wrap whole stack in calibration ---
    calibrated_stack = CalibratedClassifierCV(
        estimator=stack,
        method="isotonic",  # use "sigmoid" if data is small
        cv=5
    )

    # --- Fit on training data ---
    calibrated_stack.fit(X, y)

    # --- Apply to test set ---
    new_df = df_full.loc[df_full.index.isin(test_set)]
    X_new = new_df[X.columns]
    y_proba_new = calibrated_stack.predict_proba(X_new)[:, 1]

    # --- Create results dataframe sorted by probability ---
    results = pd.DataFrame({
        "species": new_df.index,
        "prob_positive": y_proba_new
    }).sort_values("prob_positive", ascending=False)

    # --- Confidence-band logic ---
    count_true, count_false = 0, 0
    print(results)
    for _, row in results.iterrows():
        if row.species not in species_truth_dictionary:
            species_truth_dictionary[row.species] = []

        if abs(row.prob_positive - 0.5) > 0.1:
            if row.prob_positive >= 0.5 and row.species in all_positive_species:
                species_truth_dictionary[row.species].append(1)
                count_true += 1
            elif row.prob_positive < 0.5 and row.species not in all_positive_species:
                species_truth_dictionary[row.species].append(1)
                count_true += 1
            else:
                species_truth_dictionary[row.species].append(0)
                count_false += 1
        else:
            species_truth_dictionary[row.species].append("c")

    # --- Track run-level accuracy ---
    if (count_true + count_false) > 0:
        run_accuracy = count_true / (count_true + count_false)
    else:
        run_accuracy = 0

    correct_pred_percentage_list.append(run_accuracy)
    print(run_accuracy)
    print((count_true, count_false, len(new_df.index)))


run  0


/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / l

                             species  prob_positive
6                 Cyaniris_semiargus       0.920000
26                    Plebejus_argus       0.920000
27                Polyommatus_icarus       0.920000
28                    Troides_aeacus       0.832812
18                   Melitaea_cinxia       0.828586
8                    Erebia_aethiops       0.791348
19         Papilio_dardanus_tibullus       0.785040
22                   Papilio_polytes       0.769030
0             Anthocharis_cardamines       0.763333
1   Argynnis_bischoffii_washingtonia       0.749226
17               Melinaea_menophilus       0.746667
12                  Leptidea_sinapis       0.746667
20                  Papilio_demoleus       0.696667
21                   Papilio_machaon       0.686667
14                Mechanitis_mazaeus       0.641826
9                  Euphydryas_editha       0.636667
5                   Colias_eurytheme       0.625500
13                   Maniola_jurtina       0.597312
11          

/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / l

                             species  prob_positive
12               Glaucopsyche_alexis       0.937388
4                   Bicyclus_anynana       0.936031
18                Maniola_hyperantus       0.922225
17                  Lysandra_coridon       0.920000
26                    Plebejus_argus       0.914911
8                    Erebia_aethiops       0.883333
6                 Cyaniris_semiargus       0.833333
25                       Pieris_napi       0.833333
14                   Heliconius_sara       0.833333
2   Argynnis_bischoffii_washingtonia       0.833333
27                Polyommatus_icarus       0.822280
23                    Papilio_memnon       0.803333
11                 Fabriciana_adippe       0.796233
5                  Calycopis_cecrops       0.677515
16                  Leptidea_sinapis       0.666667
0             Anthocharis_cardamines       0.570000
19                Mechanitis_mazaeus       0.570000
24                   Papilio_polytes       0.544513
30          

...

                     species  prob_positive
7            Erebia_aethiops       0.840000
10       Glaucopsyche_alexis       0.840000
15        Lysandra_bellargus       0.840000
27            Plebejus_argus       0.840000
1         Boloria_euphrosyne       0.824391
20         Nymphalis_c-album       0.813043
18          Mellicta_athalia       0.810741
5         Cyaniris_semiargus       0.779519
22            Oeneis_ivallda       0.754286
16        Maniola_hyperantus       0.754286
24            Papilio_elwesi       0.687619
30  Troides_oblongomaculatus       0.673333
11    Heliconius_charithonia       0.655209
23   Ornithoptera_alexandrae       0.652792
21     Nymphalis_polychloros       0.624054
2               Brenthis_ino       0.587619
4        Celastrina_argiolus       0.587619
26      Parnassius_glacialis       0.587619
25            Papilio_memnon       0.587619
28    Teinopalpus_imperialis       0.587619
12      Heliconius_nattereri       0.587619
9          Euphydryas_editha    

/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / l

                             species  prob_positive
9                  Hipparchia_semele       0.940000
23                    Papilio_elwesi       0.940000
15               Melanargia_galathea       0.940000
24                   Phengaris_arion       0.940000
2                 Boloria_euphrosyne       0.866341
25                  Pieris_brassicae       0.838582
14                Lysandra_bellargus       0.821309
22         Papilio_dardanus_tibullus       0.815000
19                 Nymphalis_c-album       0.790000
13                 Leptophobia_aripa       0.787385
20                      Nymphalis_io       0.705672
0             Anthocharis_cardamines       0.652820
17                  Mellicta_athalia       0.628801
28            Teinopalpus_imperialis       0.586249
1   Argynnis_bischoffii_washingtonia       0.515627
6                    Dircenna_loreta       0.475181
5                  Danaus_chrysippus       0.470261
30                    Zerene_cesonia       0.415000
4           

/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / last_over_new_count - new_sum) ** 2
/home/saurav/miniconda3/envs/genomics_work/lib/python3.11/site-packages/sklearn/utils/extmath.py:1180: RuntimeWarning: invalid value encountered in divide
  * (last_sum / l

KeyboardInterrupt: 

In [53]:
output  = "Species,True,False,Confused,Total_test"
for species in species_truth_dictionary:
    # if species not
    count_true = 0
    count_false = 0
    count_confused = 0
    for value in species_truth_dictionary[species]:
        if value == 1:
            count_true +=1
        elif value == 0:
            count_false += 1
        elif value == "c":
            count_confused += 1
        
    output  += f"\n{species},{(count_true)/len(species_truth_dictionary[species])},{(count_false)/len(species_truth_dictionary[species])},{(count_confused)/len(species_truth_dictionary[species])},{len(species_truth_dictionary[species])}"       
            
    # print(species, (count_true)/len(species_truth_dictionary[species]), (count_false)/len(species_truth_dictionary[species]),(count_confused)/len(species_truth_dictionary[species]),len(species_truth_dictionary[species] ))

In [54]:
with open(f"{location}/20.CLassification/test_200_2.csv",'w') as out_file:
    out_file.write(output)

In [55]:
print(output)

Species,True,False,Confused,Total_test
Cyaniris_semiargus,1.0,0.0,0.0,7
Plebejus_argus,1.0,0.0,0.0,12
Polyommatus_icarus,1.0,0.0,0.0,13
Troides_aeacus,0.09090909090909091,0.8181818181818182,0.09090909090909091,11
Melitaea_cinxia,1.0,0.0,0.0,7
Erebia_aethiops,1.0,0.0,0.0,12
Papilio_dardanus_tibullus,0.0,0.5,0.5,8
Papilio_polytes,0.2,0.6,0.2,10
Anthocharis_cardamines,0.7272727272727273,0.0,0.2727272727272727,11
Argynnis_bischoffii_washingtonia,0.7777777777777778,0.0,0.2222222222222222,9
Melinaea_menophilus,0.375,0.375,0.25,8
Leptidea_sinapis,0.7142857142857143,0.14285714285714285,0.14285714285714285,14
Papilio_demoleus,0.2727272727272727,0.18181818181818182,0.5454545454545454,11
Papilio_machaon,0.18181818181818182,0.2727272727272727,0.5454545454545454,11
Mechanitis_mazaeus,0.14285714285714285,0.5714285714285714,0.2857142857142857,7
Euphydryas_editha,0.36363636363636365,0.0,0.6363636363636364,11
Colias_eurytheme,0.09090909090909091,0.2727272727272727,0.6363636363636364,11
Maniola_jurtina,